
---

# Transformaciones
---


In [3]:
import pandas as pd
import ast
df = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\Movies\\movies_dataset.csv', low_memory=False)
df2 = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\Movies\\credits.csv', low_memory=False)

#'''trabajaré al mismo tiempo con el DF Credits, ya que algunas funciones de la API necesitan datos provenientes de este'''

In [4]:
#los convertimos a 'parquet' para optimizar el rendimineto
df.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy')
# Guardar con compresión snappy
df2.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy')



In [5]:
# Contar el número de filas duplicadas
duplicados = df.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable, esto simplemente para verificar que si existían filas duplicadas
numero_de_duplicados = duplicados

print("Número de filas duplicadas eliminadas:", numero_de_duplicados)

#segundo DF
# Contar el número de filas duplicadas 
duplicados = df2.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df2.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable, esto simplemente para verificar que si existían filas duplicadas
numero_de_duplicados = duplicados
print("Número de filas duplicadas eliminadas en el segundo DF:", numero_de_duplicados)

Número de filas duplicadas eliminadas: 17
Número de filas duplicadas eliminadas en el segundo DF: 37


In [6]:

'''Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.'''
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])


In [7]:
'''Los valores nulos del campo release date deben eliminarse.'''

df = df.dropna(subset=['release_date'])

In [8]:
'''Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.'''

df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)

In [9]:
'''De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.'''

#convertimos las fechas al datatype datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') 
df['release_year'] = df['release_date'].dt.year

In [10]:
'''Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.'''
# Convertir las columnas 'revenue' y 'budget' a tipo numérico
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
# Rellenar valores nulos en 'revenue' y 'budget' con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Crear la columna 'return' calculando revenue / budget
# Usar np.where para evitar división por 0
import numpy as np
df['return'] = np.where(df['budget'] == 0, 0, df['revenue'] / df['budget'])

In [12]:
''' Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una
lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API!
O bien buscar la manera de acceder a esos datos sin desanidarlos.'''
# Función para convertir cadenas a diccionarios o listas
def convertir_en_obj(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return x  # Retorna la cadena si no es un diccionario válido
    return x

# Aplicar la conversión a las series 
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convertir_en_obj)
df['genres'] = df['genres'].apply(convertir_en_obj)
df['production_companies'] = df['production_companies'].apply(convertir_en_obj)
df['production_countries'] = df['production_countries'].apply(convertir_en_obj)
df['spoken_languages'] = df['spoken_languages'].apply(convertir_en_obj)

# Desanidar diccionarios
BTC_df = df['belongs_to_collection'].apply(pd.Series)
BTC_df = BTC_df.drop(columns = 0, errors='ignore')  # Eliminar columna de índice si existe
nuevos_nombres_BTC = ['BTCid', 'BTCname', 'BTCposter_path', 'BTCbackdrop_path']
BTC_df.columns = nuevos_nombres_BTC

# Desanidar listas
genres_df = df['genres'].apply(pd.Series)
production_companies_df = df['production_companies'].apply(pd.Series)
production_countries_df = df['production_countries'].apply(pd.Series)
spoken_languages_df = df['spoken_languages'].apply(pd.Series)

# Asegurarse de que no haya nombres de columnas duplicados
genres_df = genres_df.rename(columns=lambda x: f'genre_{x}')
production_companies_df = production_companies_df.rename(columns=lambda x: f'company_{x}')
production_countries_df = production_countries_df.rename(columns=lambda x: f'country_{x}')
spoken_languages_df = spoken_languages_df.rename(columns=lambda x: f'language_{x}')

# Eliminar las columnas originales
df = df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'])

# Unir los DataFrames desanidado al DataFrame original
df = df.join(BTC_df)
df = df.join(production_companies_df)
df = df.join(production_countries_df)
df = df.join(spoken_languages_df)


C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:21: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  BTC_df = df['belongs_to_collection'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:27: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  genres_df = df['genres'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:28: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  production_companies_df = df['production_companies'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:29: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series


---

### Transformaciones sobre credits (API)
---


In [13]:
''' Al ser un DF muy grande y necesitar solo partes muy especificas de los diccionarios que componen las series del DF,
en vez de desanidarlo todo, utilizaremos otras tecnicas para extraer los datos que necesitamos, y con estos crearemos una nueva tabla,
esta misma contendra solamente los datos que necesitamos para las funciones de nuestra API '''

# Extraer los nombres de la columna 'cast'
names = df2['cast'].apply(lambda x: [item['name'] for item in ast.literal_eval(x) if 'name' in item])

# Convertir la serie 'names' en un DataFrame
names_df = pd.DataFrame(names.tolist())

# Añadir la palabra 'nombre' delante del nombre de cada serie
names_df.columns = ['nombre_' + str(col) for col in names_df.columns]

# Crear una serie que contenga únicamente el nombre del director
names_df['Nombre_Director'] = df2['crew'].apply(
    lambda x: next((item['name'] for item in ast.literal_eval(x) if item.get('job') == 'Director'), None)
)

# volvemos a añadir los ID a las filas, ya que los necesitaremos luego para hacer un "Join"
names_df['id'] = df2['id'].astype(int)





In [14]:
df['id'] = pd.to_numeric(df['id'], errors='coerce')


---

### Exportación de los Datafames

---


In [63]:
df.to_parquet('transformados.parquet', index=False)


#estas transformaciones las hago ya habiendo casi finalizado el trabajo, para el sistema de recomendación
#He decidido que para este crearé un nuevo DF, el cual será una versión reducida del mismo, y que mantendrá unicamente las series que considere pertinentes

---

### Transformaciones para Machine learning

---


In [107]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
df = pd.read_parquet('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\transformados.parquet')


In [108]:
#por cuestiones de rendimiento deberemos reducir realmente mucho el numero de filas y series, no pudiendo usar series que contengan strings
#tambien reduciremos muchisimo el numero de filas, para que el archivo final no supere el peso de 512 mb impuesto por render
#vamos a crear nuestra matriz de similitud del coseno y luego importarla a nuestro "main.py"
series_ML =['title','popularity', 'runtime', 'vote_average']
df = df[series_ML]
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

In [109]:

# Crear una columna que cuenta el número de ceros y NaNs por fila
df['count_zeros_nans'] = (df == 0).sum(axis=1) + df.isna().sum(axis=1)

# Ordenar el DataFrame por esta nueva columna en orden descendente
df = df.sort_values(by='count_zeros_nans', ascending=False)

# Eliminar filas con más valores 0 o NaN
df = df.iloc[len(df) // 2:]
df = df.iloc[len(df) // 2:]
df = df.iloc[len(df) // 2:]
df = df.iloc[len(df) // 2:]
df = df.iloc[len(df) // 2:]


# Eliminar la columna auxiliar 'count_zeros_nans'
df = df.drop(columns=['count_zeros_nans'])


# Reemplazar 0 por NaN en todas las columnas excepto 'title'
df.loc[:, df.columns != 'title'] = df.loc[:, df.columns != 'title'].replace(0, np.nan)

# Imputar NaNs con el promedio de cada columna (excepto 'title')
df.loc[:, df.columns != 'title'] = df.loc[:, df.columns != 'title'].fillna(df.mean(numeric_only=True))


In [110]:
df.reset_index(drop=True, inplace=True)

In [111]:
df

,title,popularity,runtime,vote_average
0,Torch Song,0.574971,90.0,5.4
1,Of Human Hearts,0.404973,103.0,2.3
2,Princess O'Rourke,0.319906,94.0,6.5
3,The Whisperers,0.470596,105.0,6.6
4,Shogun's Ninja,0.977162,117.0,5.3
...,...,...,...,...
1416,Cyrus,10.420010,91.0,6.1
1417,The Human Centipede (First Sequence),12.845850,92.0,4.9
1418,The Ten Commandments,1.725825,136.0,7.0
1419,This Is the Army,0.800156,121.0,8.3


In [112]:
# Extraer características numéricas para las recomendaciones
features = df[['popularity', 'runtime', 'vote_average']]

# Normalizar las características
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Calcular similitudes usando el coseno
cosine_sim = cosine_similarity(features_scaled)

In [113]:

np.save('similitud_del_coseno.npy', cosine_sim)